<a href="https://colab.research.google.com/github/slesarev-hub/pixel-cnn/blob/master/pixel_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install --ignore-installed --upgrade tensorflow==2.0

In [0]:
!python -c 'import tensorflow as tf; print(tf.__version__)' 

2.0.0


In [0]:
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/slesarev-hub/pixel-cnn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'pixel-cnn' already exists and is not an empty directory.


In [0]:
%cd pixel-cnn
#!git checkout -b ahe 5de8cecf6c41b56f58cdd85d1aae1450c27673de

/content/pixel-cnn


In [0]:
!python train.py --data_dir . -d ahe -b 1 -u 1 -t 5 --load_params 80 --ckpt_folder_drive_dir '/content/drive/My Drive/pixel-cnn'

In [0]:
!python train.py --data_dir . -d ahe -b 1 -u 1 -t 5 --ckpt_folder_drive_dir '/content/drive/My Drive/pixel-cnn'

In [0]:
!python3 -c 'import tensorflow as tf; print(tf.__version__)'
!python train.py --data_dir . --save_dir . -d ahe -b 1 -u 1 -t 5 -l 0 

**1.**

![alt text](https://cdn-images-1.medium.com/freeze/max/1000/1*ToPT8jnb5mtnikmiB42hpQ.png?q=20)

**Softmax** (activation function)

$\sigma(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}} \text{ for } i = 1, \dotsc , K \text{ and } \mathbf z=(z_1,\dotsc,z_K) $

**2.**

**Softmax** - ignore relationship betwen numbers such as 220 and 221; even worse, if a pixel value never presents in the training dataset, it won’t be predicted in the testing dataset either. A better way is predict the distribution of pixel values, so every value will a have probility to present when generating images

 A mixture of distriction is a linear combination of a number of distributions: $p(x) = a_1 * p_1(x) + a_2 * p_2(x) + … a_n * p_n(x)$

[source](https://medium.com/@smallfishbigsea/an-explanation-of-discretized-logistic-mixture-likelihood-bdfe531751f0)

![alt text](http://cs231n.github.io/assets/nn1/neuron_model.jpeg)



**3.**

![](https://stanford.edu/~shervine/teaching/cs-230/illustrations/convolution-layer-a.png)

In [0]:
http://cs231n.github.io/assets/conv-demo/index.html




impractical to connect neurons to all neurons in the previous volume. Instead, we will connect each neuron to only a local region of the input volume. **Receptive field** of the neuron (equivalently this is the filter size)

**Stride** - count of pixels which we slide the filter 

The most common **downsampling** operation is max, giving rise to max pooling, here shown with a stride of 2. That is, each max is taken over 4 numbers

![alt text](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

**4.**

**Layer types**

*INPUT* - will hold the raw pixel values of the image

*CONV* - compute the output of neurons that are connected to local regions in the input, each computing a dot product between their weights and a small region they are connected to in the input volume. 

*RELU* - apply an elementwise activation function, leaves the size of the volume unchanged 

*POOL* - perform a **downsampling** operation along the spatial dimensions (width, height)

*FC* - (i.e. fully-connected) compute the class scores, resulting in volume of size [1x1x10], where each of the 10 numbers correspond to a class score


**RestNet**

* with the network depth increasing, accuracy gets  saturated
* explicitly let these layers fit a residual mapping (F(x)+x)

![alt text](https://neurohive.io/wp-content/uploads/2019/01/resnet-e1548261477164.png)

**PixelCNN++ scheme**

6 blocks of 5 ResNet layers

![alt text](https://asset-pdf.scinapse.io/prod/2581236139/figures/figure-2.jpg)

**Deconvolution (upsampling)**

possible ways:

![](https://i.stack.imgur.com/YyCu2.gif)
![](https://i.stack.imgur.com/f2RiP.gif)

**5.**

**Droput**

* given many more features than examples, linear models can overfit

*  want to expect that adding some random noise to the pixels should be mostly harmless

* when training deep network with many layers, enforcing smoothness just on the input-output mapping misses out on what is happening internally in the network [\[Srivastava et al., 2014\]](http://jmlr.org/papers/volume15/srivastava14a.old/srivastava14a.pdf)

* Throughout training, on each iteration, dropout regularization consists simply of zeroing out some fraction of the nodes in each layer before calculating the subsequent layer

![alt text](https://www.depends-on-the-definition.com/wp-content/uploads/2019/08/dropout.jpeg)

**PixelCNN++** vs **PixelCNN**

*  Discretized logistic mixture likelihood (2) on the pixels, rather than a 256-way 
softmax (1), which we find to speed up training

The standard PixelCNN model specifies the conditional distribution of a sub-pixel, or color channel
of a pixel, as a full 256-way softmax. This gives the model a lot of flexibility, but it is also very costly
in terms of memory.

In PixelCNN++ assume there is
a latent color intensity $\nu$ with a continuous distribution, which is then rounded to its nearest 8-bit
representation to give the observed sub-pixel value x. Distribution -  mixture of logistic distributions which allows us to easily calculate the probability on the observed discretized value x. For all sub-pixel values x excepting the edge cases 0 and 255 we have:

$\nu \sim \sum_{i=1}^K\pi_ilogistic(\mu_i, s_i)$

$P(x|π, µ, s) = \sum_{i=1}^K\pi_i[\sigma((x + 0.5 − \mu_i)/s_i) − \sigma((x − 0.5 − \mu_i)/s_i)]$ 

$K = 5$

*  Condition on whole pixels, rather than R/G/B sub-pixels,
simplifying the model structure

PixelCNN factorizes the generative model over 3 sub-pixels. This
allows for very general dependency structure. 
PixelCNN++:
  
$~~$firstly predict the red channel using a discretized mixture of logistics as described before


$~~$secondly predict the green channel using a predictive distribution of the
same form. Here we allow the means of the mixture components to linearly depend on the value of
the red sub-pixel

$~~$finally model the blue channel in the same way, where we again only allow linear dependency on the red and green channels.
*  Use downsampling to efficiently capture
structure at multiple resolutions

*  Introduce additional short-cut connections to further speed up optimization

*PixelCNN* only uses convolutions with small **receptive field** (3) (good at capturing local dependencies, but not at modeling long range structure).

*PixelCNN++* to model whole structure - use **downsampling** by using convolutions of **stride** $2$. (loses information, but can compensate for this by introducing additional short-cut connections
into the network)

*  Regularize the model using dropout

In *PixelCNN++* - standard binary dropout on the residual path after the first
convolution